<a href="https://colab.research.google.com/github/vadhri/ai-notebook/blob/main/he/gm_scheme_xor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import secrets
import math
from sympy import legendre_symbol


### Goldwasser Micali Encryption Scheme ( GM scheme )

Introduced semantic security, multi-fold encryption produces different cipher texts.

Scheme has 3 parts as below.
- Probalisitc key generation algorithm
- Probabilistic Encryption Algorithm
- Deterministic Decryption Algorithm

#### Key generation

The process of generation of key has the steps below which is same as RSA

https://en.wikipedia.org/wiki/RSA_cryptosystem

- Miller–Rabin primality test and prime generation (generate_prime)
- Legendre symbol via Euler's criterion (legendre_symbol)
- Jacobi symbol (jacobi_symbol)
- Finding a quadratic non-residue x such that (x|p) = (x|q) = -1 (find_non_residue)
- keygen(bits, use_blum) which returns public_key=(x,N) and secret_key=(p,q).
  - If use_blum=True it generates Blum primes (p ≡ q ≡ 3 (mod 4)) and uses x = N-1.

In [ ]:
# choose 2 random prime numbers between min and max below

min_primes = 2**99
max_primes = 2**100

1. Define a numeric range for primes — between 2^49 and 2^50.
    - Ensures both primes are large and roughly equal in magnitude.

2. Randomly sample numbers within that range using the secrets library.
    - Provides cryptographically secure randomness (unpredictable by attackers).
    - random module is not suitable, since its sequence can be reproduced if the seed is known — exposing all generated primes.

3. Skip even or trivially composite numbers quickly for efficiency.

4. Apply the Miller–Rabin probabilistic primality test to check if a number behaves like a prime.
    - Uses modular exponentiation to rule out composites without full factorization.
    - Repeats the test with multiple random bases for reliability.

5. Accept the first number that passes all tests as a probable prime.

6. Repeat the process until you have two distinct primes (for GM or RSA modulus).

In [ ]:
# find all primes between min and max
"""(generated by AI) Miller-Rabin primality test."""
def is_probable_prime(n, k=10):
    if n < 2:
        return False
    small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
    for p in small_primes:
        if n % p == 0:
            return n == p

    d, s = n - 1, 0
    while d % 2 == 0:
        d //= 2
        s += 1

    for _ in range(k):
        a = secrets.randbelow(n - 3) + 2
        x = pow(a, d, n)
        if x in (1, n - 1):
            continue
        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


"""(generated by AI) Generate a random probable prime in [low, high]."""
def generate_prime_in_range(low, high):
    assert low < high
    while True:
        n = secrets.randbelow(high - low) + low
        if n % 2 == 0:
            n += 1  # make it odd
        if is_probable_prime(n):
            return n

In [ ]:
# generate a pair of primes numbers.
for test in range(10):
    q = p = generate_prime_in_range(min_primes, max_primes)
    while q == p:
      q = generate_prime_in_range(min_primes, max_primes)
      print(p, q)

1069278847378778461099214301467 1119922288502061916434895119283
659122379706589765166335603021 766470634330195885283849274197
840775441600482876482910156113 746664566184964796757791938919
1005275729619434977237838701897 971912808177914923035424498753
854967281049047903746469189291 806121250725578168197357819007
773071975718983859179524869611 1248340980434768858908784168071
1048671382418090085665408155081 995610504442237916410315143443
1070440217539110015835007525767 1000483489302939673396702726511
981434225753327585200898858017 744371297482661495519919408347
639143728359782880674451785197 1205904515042966536754709683851


In [ ]:
# (Generated by AI)
"""Jacobi symbol for completeness (optional)."""
def jacobi_symbol(a, n):
    a = a % n
    result = 1
    while a != 0:
        while a % 2 == 0:
            a //= 2
            if n % 8 in (3, 5):
                result = -result
        a, n = n, a
        if a % 4 == n % 4 == 3:
            result = -result
        a %= n
    return result if n == 1 else 0

# (Generated by AI)
"""Find x such that (x|p) = (x|q) = -1."""
def find_non_residue(p, q):
    N = p * q
    while True:
        x = secrets.randbelow(N - 2) + 2  # random 2 ≤ x ≤ N−1
        if pow(x, (p - 1) // 2, p) == p - 1 and pow(x, (q - 1) // 2, q) == q - 1:
            # Optional check
            assert jacobi_symbol(x, N) == 1
            return x


In [ ]:
print (f"p = {p}, q={q}")
quadratic_non_residue = find_non_residue(p, q)
p_res =  pow(quadratic_non_residue, (p - 1) // 2, p)
q_res =  pow(quadratic_non_residue, (q - 1) // 2, q)
print (f"x = {quadratic_non_residue}, (x|p) = {p_res}, (x|q) = {q_res}")
print (p_res == -1 % p, q_res == -1 % q )

p = 639143728359782880674451785197, q=1205904515042966536754709683851
x = 3052074619730181529190760787741758610591691881896692315090, (x|p) = 639143728359782880674451785196, (x|q) = 1205904515042966536754709683850
True True


In [ ]:
N = p * q
x = quadratic_non_residue
public_key = (x, N)
private_key = (p, q)

print (f"public_key = {public_key}, \nprivate_key = {private_key}")

public_key = (3052074619730181529190760787741758610591691881896692315090, 770746307790457512721140446179316162020748732050770431753647), 
private_key = (639143728359782880674451785197, 1205904515042966536754709683851)


#### Message encryption

In [ ]:
message = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."
cipher = []

m_binary = "".join([format(ord(i), '08b') for i in message])

for bit in m_binary:
  y = secrets.randbelow(N - 2) + 2
  while (math.gcd(y, N) != 1):
    y = secrets.randbelow(N - 2) + 2

  ci = (y**2 * pow(x, int(bit))) % N
  cipher.append(ci)
print (cipher)


[...]


#### Message decryption

In [ ]:
decryption = []

for c in cipher:
  p_res =  pow(c, (p - 1) // 2, p)
  q_res =  pow(c, (q - 1) // 2, q)
  if (p_res == -1 % p and q_res == -1 % q ):
    m = 1
  else:
    m = 0

  decryption.append(m)

#group packs of 8 and turn them to ascii
decryption_ascii = []
for i in range(0, len(decryption), 8):
    byte = decryption[i:i+8]
    ascii_value = int("".join(map(str, byte)), 2)
    decryption_ascii.append(chr(ascii_value))

decrypted = "".join(decryption_ascii)
print ("Is decryption same as original message ? ", decrypted == message)
print ("Lengths of texts = ", len(decrypted), len(message))
print (decrypted)

Is decryption same as original message ?  True
Lengths of texts =  574 574
Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.


### XOR implemenation on encrypted data

In [ ]:
decryption = []

def encrypt(message):
    cipher = []
    bits = bin(message)[2:].zfill(32)

    for bit in bits:
        y = secrets.randbelow(N - 2) + 2
        while math.gcd(y, N) != 1:
            y = secrets.randbelow(N - 2) + 2
        c = (pow(y, 2, N) * pow(x, int(bit), N)) % N
        cipher.append(c)
    return cipher

def decrypt(cipher):
    bits = []
    for c in cipher:
        bit = 0 if legendre_symbol(c, p) == 1 else 1
        bits.append(str(bit))
    return int("".join(bits), 2)

for i in range(100):
  a = secrets.randbelow(2**32)
  b = secrets.randbelow(2**32)
  e_a = encrypt(a)
  e_b = encrypt(b)
  xor_cipher = [(ea * eb) % N for ea, eb in zip(e_a, e_b)]
  decrypted = decrypt(xor_cipher)
  print(f"a = {a}, b = {b}, decrypted = {decrypted}, {decrypted == a^b}")



a = 3512289452, b = 3854130920, decrypted = 887100996, True
a = 146272976, b = 528337961, decrypted = 399130361, True
a = 77461710, b = 2063751426, decrypted = 2141168076, True
a = 526338536, b = 305220386, decrypted = 225313482, True
a = 31447529, b = 3598468075, decrypted = 3617823234, True
a = 69806098, b = 3453691671, decrypted = 3388083973, True
a = 15362995, b = 1053217669, decrypted = 1043118134, True
a = 3831246242, b = 3470812502, decrypted = 716982516, True
a = 2259705644, b = 3878817777, decrypted = 1635930333, True
a = 3236546890, b = 3765451948, decrypted = 546960870, True
a = 2373633252, b = 563572530, decrypted = 2901257174, True
a = 2259111322, b = 922744031, decrypted = 2958604613, True
a = 171640412, b = 443333437, decrypted = 274185569, True
a = 3444966290, b = 2729711914, decrypted = 1877088952, True
a = 3736377927, b = 1047477799, decrypted = 3772490848, True
a = 4133898806, b = 4243216604, decrypted = 176952042, True
a = 3723273674, b = 625646171, decrypted = 4171

In [ ]:
a,b, a^b

(1346323799, 3171598167, 3979737600)